In [18]:
import json
jsonl_file = "first_100_response.jsonl"
from datasets import load_dataset

vali_dataset = load_dataset('alexshengzhili/SciCapInstructed-graph-only-qa', split='1_percent_as_validation[:100]')
data = vali_dataset.filter(lambda x: x['q_a_pairs'] is not None and len(x['q_a_pairs']) > 0)


answers = []
with open(jsonl_file, 'r') as f:
    for line in f:
        example = json.loads(line)
        answers.append(example)


Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapInstructed-graph-only-qa-c5897d2f1995d1be/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapInstructed-graph-only-qa-c5897d2f1995d1be/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-f97ab97c5880d73a.arrow


In [16]:
groundtruth['q_a_pairs'][0]

[['What is the difference between the two graphs in Figure 10?',
  'The two graphs in Figure 10 show the performance of still-to-video FR systems on the Chokepoint and COX-S2V databases, respectively. The x-axis of both graphs shows the number of synthetic ROIs generated under both pose and lighting effects, while the y-axis shows the average AUC and AUPR.'],
 ['What are the key observations that can be made from the graphs?',
  'The key observations that can be made from the graphs are as follows:\n\n* The performance of both still-to-video FR systems improves as the number of synthetic ROIs increases.\n* The performance of the still-to-video FR system with the SRC classifier is better than the performance of the still-to-video FR system with the SVM classifier.\n* The performance of the still-to-video FR system on the Chokepoint database is better than the performance of the still-to-video FR system on the COX-S2V database.']]

In [17]:
answers[0]

{'question_id': 0,
 'prompt': 'What is the difference between the two graphs in Figure 10?',
 'text': 'In Figure 10, there are two different graphs showing educational attainment for men and women in the United States. The first graph, which is a bar graph, displays the number of men with degrees as the vertical axis and the number of degrees as the horizontal axis. The second graph, which is a line graph, shows the trend of women earning degrees over time. The two graphs present different aspects of educational attainment, providing insights into how men and women have pursued higher education in the past and how the trend has evolved over time.',
 'answer_id': 'aauN8seAY4GK8WUG5UGuAC',
 'model_id': 'alexshengzhili/llava-7b-v0',
 'metadata': {}}

In [26]:
import argparse
import json
import os

import openai
import tqdm
import time
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_KEY")
system_message = """
You are a helpful and precise assistant for checking the quality of the answer.
You are given the graph's caption, the context of the graph, the abstract, tthe title

And then you are given the question, the reference answer, and the answer generated by the model. Please
think about how helpful the model answer is to the user and rate the model answer on a scale of 0 to 10, 
where 0 is not helpful at all and 10 is very helpful. Just return the floating number between 0 and 10.
"""

def construct_input_string(index):
    content = dict()
    cur_example = data[index]
    content['title'] = cur_example['title']
    content['abstract'] = cur_example['abstract']
    content['caption'] = cur_example['caption']
    content['Question to the model'] = cur_example['q_a_pairs'][0][0]
    content['reference_answer'] = cur_example['q_a_pairs'][0][1]
    content['Candidate model answer'] = answers[index]['text']
    return json.dumps(content)


def get_openai_response(content_string):
    openai_response = openai.ChatCompletion.create(
                    model='gpt-4',
                    messages=[{
                        'role': 'system',
                        'content': system_message
                    }, {
                        'role': 'user',
                        'content': content_string
                    }],
                    temperature=0.2,  # TODO: figure out which temperature is best for evaluation
                    max_tokens=500,
                )['choices'][0]['message']['content']
    return openai_response

openai_responses = []
for i in range(len(data)):
    content_string = construct_input_string(i)
    openai_response = get_openai_response(content_string)
    print(openai_response)
    openai_responses.append(openai_response)
    time.sleep(2)


0.0
0.0
0.0
5.0
0.0
3.0
3.0
0.0
0
9.5
0.0
0
0
0.0
0.0
5.0
3.0
4.5
0.0
0
0.0
4.5
2.0
0
3.0
8.5
3.0
6.5
0
2.0
0.0
0.0
0.0
4.5
5.0
0
6.5
3.0
2.0
0.0
4.5
0.0
3.5
7.5
0.0
0.0
9.5
5.5
2.0
1.0
2.0
4.5
2.0
0.0
2.0
2.0
0.0
4.5
4.5
4.5
0.0
3.0
3.0
0.0
9.5
4.5
2.0
3.0
0
0.0
2.0
0
2.0
2.0
2.0
0
0
5.0
2.0
0
0
2.0
0.0
3.0
0.0
3.5
2.0
2.0
7.5
1.0
0.0
2.0
0.0
0.0
0.0
9.5
3.0
6.5
5.5
0.0


In [28]:
import numpy as np
openai_responses_float = [float(str) for str in openai_responses]
np.mean(openai_responses_float), np.std(openai_responses_float)

(2.305, 2.6104549411932014)

In [29]:
response_model = [item['text'] for item in answers]

In [31]:
new_data = data.add_column("llava_answer", response_model)
rated_data = new_data.add_column("openai_rating", openai_responses_float)
file_path = "llava_answer_and_openai_rating_first_100.jsonl"
with open(file_path, 'w') as f:
    for example in rated_data:
        json_str = json.dumps(example)
        f.write(json_str + '\n')